In [3]:
from __future__ import print_function

import sys
from operator import add
import pyspark
import os
import re
import numpy as np

def square_delta(lines):
    re1 = '[+-]?\d+\.\d+'
    re2 = '[+-]?\d+\.'
    numbers = re.compile("(%s|%s)" % (re1, re2)).findall(lines)
    return np.square(float(numbers[0])-float(numbers[1]))
    

if __name__ == "__main__":
    input_path = "/mnt/wiktorskit-jungwonseo-ns0000k/home/notebook/group03/Renny-temp/output01/predictions_whole_3000.txt"
    output_path = "/mnt/wiktorskit-jungwonseo-ns0000k/home/notebook/group03/Renny-temp/RMSE01"
    
    if not os.path.isfile(input_path) or "/mnt/" not in input_path :
        print("Please check your input path again")
        sys.exit(-1)

    sc = pyspark.SparkContext.getOrCreate()

    text_file = sc.textFile("file://"+input_path)
    RMSE = text_file.map(lambda line: ('RMSE',[1,square_delta(line)]))\
           .reduceByKey(lambda a, b :[a[0] + b[0], a[1]+b[1]]).map(lambda rmse : ('RMSE',np.sqrt(rmse[1][1]/rmse[1][0])))
    
    RMSE.saveAsTextFile(output_path)
    sc.stop()
